# LIBRERIAS


In [1]:

import pandas as pd
import os
import datetime as dt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import matplotlib
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression , LinearRegression, Lasso, Ridge, ElasticNet, TweedieRegressor 
from sklearn.svm import SVC
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix, roc_auc_score, roc_curve, auc
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
import matplotlib.pyplot as plt
import numpy as np
from itertools import cycle
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import NearMiss, EditedNearestNeighbours, TomekLinks
from collections import Counter  
from imblearn.datasets import make_imbalance  
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.metrics import classification_report_imbalanced
from imblearn.pipeline import make_pipeline
from time import time
import packages.modelado as MD

/tmp/ipykernel_2440/1759693357.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# CARGA DE DATOS

In [2]:
df=pd.read_csv("./Datos/Transformados/df_transformado.csv")

In [6]:
df.head()

,asset,asset_type,brand,segment,checkin_time,checkout_time,booking_time,adult_count,child_count,origin,...,rangos_niños,rango_de_numero_niños,cancelled_imp,dif_can_in,dif_checkin_cancelacion,dif_can_res,dif_fecha_cancelacion_booking,dif_ent_res,dif_reserva_entrada,duracion_reserva
0,Líbere Bilbao La Vieja,apartment,libere,Leisure Individual,2022-02-01 14:00:00+00:00,2022-02-02 10:00:00+00:00,2022-01-31 08:15:07.111000+00:00,2,0,channel_manager,...,sin niños,sin niños,2022-02-02 09:43:17.837000+00:00,0,0,3,3,2,2,1
1,Líbere Bilbao La Vieja,apartment,libere,Leisure Individual,2022-02-01 14:00:00+00:00,2022-02-02 08:56:06+00:00,2022-01-31 16:03:30.912000+00:00,2,0,direct_channel,...,sin niños,sin niños,2000-01-01 00:00:00+00:00,0,0,0,0,1,1,1
2,Líbere Bilbao La Vieja,apartment,libere,Leisure Individual,2022-02-01 14:00:00+00:00,2022-02-04 10:00:00+00:00,2022-01-31 17:14:01.468000+00:00,2,0,channel_manager,...,sin niños,sin niños,2000-01-01 00:00:00+00:00,0,0,0,0,1,1,3
3,Líbere Bilbao La Vieja,apartment,libere,Leisure Individual,2022-02-01 14:00:00+00:00,2022-02-02 10:00:00+00:00,2022-02-01 22:57:48.169000+00:00,2,0,channel_manager,...,sin niños,sin niños,2000-01-01 00:00:00+00:00,0,0,0,0,0,0,1
4,Líbere Bilbao La Vieja,apartment,libere,All Iron Staff,2022-02-02 14:00:00+00:00,2022-02-04 10:00:00+00:00,2022-02-01 15:31:48.464000+00:00,1,0,email,...,sin niños,sin niños,2000-01-01 00:00:00+00:00,0,0,0,0,1,1,2


In [7]:
df.columns

Index(['asset', 'asset_type', 'brand', 'segment', 'checkin_time',
       'checkout_time', 'booking_time', 'adult_count', 'child_count', 'origin',
       'travel_agency_name', 'requested_category', 'category_name',
       'rate_type', 'rate_name', 'nationality', 'age', 'recurrence',
       'product_count', 'reservation_net_value',
       'reservation_nights_net_value', 'status', 'cancellation',
       'cancelled_at', 'age_nan', 'extras_net_value', 'extras', 'rangos_age',
       'rangos_edad', 'total_personas', 'rangos_adultos',
       'rango_de_numero_adutlos', 'rangos_niños', 'rango_de_numero_niños',
       'cancelled_imp', 'dif_can_in', 'dif_checkin_cancelacion', 'dif_can_res',
       'dif_fecha_cancelacion_booking', 'dif_ent_res', 'dif_reserva_entrada',
       'duracion_reserva'],
      dtype='object')

In [3]:
clima=pd.read_csv("./Datos/Transformados/df_clima.csv")

In [9]:
clima.head()

,dia_mes,temp_max,temp_min,clima,fecha,ciudad
0,1,12,7,Parcialmente nublado,2022-02-01,bilbao
1,2,11,6,Lluvias ligeras a intervalos,2022-02-02,bilbao
2,3,13,5,Parcialmente nublado,2022-02-03,bilbao
3,4,13,8,Llovizna,2022-02-04,bilbao
4,5,12,8,Llovizna,2022-02-05,bilbao


# Juntar datos clima

In [4]:
df['checkin_time']=pd.to_datetime(df['checkin_time'])
df['mes']=df['checkin_time'].dt.month
df['dia_mes']=df['checkin_time'].dt.day

In [5]:
df=MD.ciudades(df,'asset')

In [6]:
clima['fecha']=pd.to_datetime(clima['fecha'])
clima['mes']=clima['fecha'].dt.month

In [7]:
clima=MD.estado_clima(clima, 'clima')

In [8]:
df_juntado=df.merge(clima, on=["ciudad", "mes", "dia_mes"], how="inner")

In [15]:
print(df_juntado.shape)
print(df.shape)
print(clima.shape)

(10574, 50)
(10574, 45)
(750, 8)


In [16]:
df_juntado.head()

,asset,asset_type,brand,segment,checkin_time,checkout_time,booking_time,adult_count,child_count,origin,...,dif_reserva_entrada,duracion_reserva,mes,dia_mes,ciudad,temp_max,temp_min,clima,fecha,estado
0,Líbere Bilbao La Vieja,apartment,libere,Leisure Individual,2022-02-01 14:00:00+00:00,2022-02-02 10:00:00+00:00,2022-01-31 08:15:07.111000+00:00,2,0,channel_manager,...,2,1,2,1,bilbao,12,7,Parcialmente nublado,2022-02-01,nublado
1,Líbere Bilbao La Vieja,apartment,libere,Leisure Individual,2022-02-01 14:00:00+00:00,2022-02-02 08:56:06+00:00,2022-01-31 16:03:30.912000+00:00,2,0,direct_channel,...,1,1,2,1,bilbao,12,7,Parcialmente nublado,2022-02-01,nublado
2,Líbere Bilbao La Vieja,apartment,libere,Leisure Individual,2022-02-01 14:00:00+00:00,2022-02-04 10:00:00+00:00,2022-01-31 17:14:01.468000+00:00,2,0,channel_manager,...,1,3,2,1,bilbao,12,7,Parcialmente nublado,2022-02-01,nublado
3,Líbere Bilbao La Vieja,apartment,libere,Leisure Individual,2022-02-01 14:00:00+00:00,2022-02-02 10:00:00+00:00,2022-02-01 22:57:48.169000+00:00,2,0,channel_manager,...,0,1,2,1,bilbao,12,7,Parcialmente nublado,2022-02-01,nublado
4,Líbere Bilbao Museo,apartment,libere,Leisure Individual,2022-02-01 14:00:00+00:00,2022-02-03 08:25:43+00:00,2022-01-26 08:25:50.021000+00:00,1,0,channel_manager,...,7,2,2,1,bilbao,12,7,Parcialmente nublado,2022-02-01,nublado


In [17]:
df_juntado.columns

Index(['asset', 'asset_type', 'brand', 'segment', 'checkin_time',
       'checkout_time', 'booking_time', 'adult_count', 'child_count', 'origin',
       'travel_agency_name', 'requested_category', 'category_name',
       'rate_type', 'rate_name', 'nationality', 'age', 'recurrence',
       'product_count', 'reservation_net_value',
       'reservation_nights_net_value', 'status', 'cancellation',
       'cancelled_at', 'age_nan', 'extras_net_value', 'extras', 'rangos_age',
       'rangos_edad', 'total_personas', 'rangos_adultos',
       'rango_de_numero_adutlos', 'rangos_niños', 'rango_de_numero_niños',
       'cancelled_imp', 'dif_can_in', 'dif_checkin_cancelacion', 'dif_can_res',
       'dif_fecha_cancelacion_booking', 'dif_ent_res', 'dif_reserva_entrada',
       'duracion_reserva', 'mes', 'dia_mes', 'ciudad', 'temp_max', 'temp_min',
       'clima', 'fecha', 'estado'],
      dtype='object')

# TRANSFORMACIONES

In [9]:
df1=MD.dummys(df_juntado,"rangos_age")
df1=MD.dummys(df1,"segment")
df1=MD.dummys(df1,"rate_type")
df1=MD.dummys(df1,"estado")
df1=MD.dummys(df1, "brand")
df1=MD.dummys(df1, "asset_type")

In [19]:
df1.columns

Index(['asset', 'asset_type', 'brand', 'segment', 'checkin_time',
       'checkout_time', 'booking_time', 'adult_count', 'child_count', 'origin',
       'travel_agency_name', 'requested_category', 'category_name',
       'rate_type', 'rate_name', 'nationality', 'age', 'recurrence',
       'product_count', 'reservation_net_value',
       'reservation_nights_net_value', 'status', 'cancellation',
       'cancelled_at', 'age_nan', 'extras_net_value', 'extras', 'rangos_age',
       'rangos_edad', 'total_personas', 'rangos_adultos',
       'rango_de_numero_adutlos', 'rangos_niños', 'rango_de_numero_niños',
       'cancelled_imp', 'dif_can_in', 'dif_checkin_cancelacion', 'dif_can_res',
       'dif_fecha_cancelacion_booking', 'dif_ent_res', 'dif_reserva_entrada',
       'duracion_reserva', 'mes', 'dia_mes', 'ciudad', 'temp_max', 'temp_min',
       'clima', 'fecha', 'estado', '+80', '16-18', '18-24', '25-35', '35-50',
       '50-65', '65-80', 'All Iron Staff', 'Business Group',
       'Business

In [10]:
y=df1.loc[:,'cancellation']
x=df1.loc[:,['product_count','recurrence','age_nan', 'duracion_reserva', '+80', '16-18', '18-24', '25-35', '35-50', '50-65', '65-80',
       'All Iron Staff', 'Business Group', 'Business Individual',
       'Leisure Group', 'Leisure Individual', 'Mid-Long Stay', 'Travel Group',
       'Travel Individual', 'flexible', 'non_refundable', 'other','lluvia',
       'lluvias_fuertes', 'niebla', 'nieve', 'nublado', 'otro', 'soleado',
       'tormenta','libere', 'naitly','aparthotel', 'apartment'
       ]]

# 

In [11]:
xtr, xte, ytr, yte = train_test_split(x, y, test_size=0.30, random_state=0, shuffle=True, stratify=y) 

In [12]:
df_train = pd.concat([xtr, ytr], axis = 1)
df_train.to_csv('./Datos/Transformados/df_train.csv', index = False)

In [13]:
df_test = pd.concat([xte, yte], axis = 1)
df_test.to_csv('./Datos/Transformados/df_test.csv', index = False)

# Modelos e Parametros

In [13]:
models_list=[LogisticRegression(max_iter=10000),MultinomialNB(),GaussianNB(),BernoulliNB(), SVC()] 

In [14]:
model_hyperparameters = {
    'log_reg':{
        'C':[1,5,10,20]
    },
    'mult_nb':{
       'alpha':[1,2,3,4,5,6,7,8,9,10],
        'fit_prior':[True, False]
     },
    'gaus_nb':{
        'var_smoothing':[1e-9, 0.5, 1,2,3,4,5,6,7,8,910]
    },
    'ber_nb':{
         'alpha':[1,2,3,4,5,6,7,8,9,10],
         'fit_prior':[True, False]
    },
    'svc':{
        'kernel':['rbf'],
       'C':[1,2,3]}
}

   

In [15]:
RANDOM_STATE = 0

# SELECCION DE MODELOS


In [17]:
sin_balanceo=MD.modelo_sin_balanceo(xtr, ytr, models_list, model_hyperparameters)

DATOS DESBALANCEADOS
False    5735
True     1666
Name: cancellation, dtype: int64
{'model_used': LogisticRegression(max_iter=10000), 'highest_score': 0.9432512728799022, 'best hyperparameters': {'C': 1}, 'balanceo': 'sin_balanceo', 'segundos': 6.162464380264282, 'hora_subida': datetime.datetime(2023, 1, 21, 23, 50, 46, 591354)}
{'model_used': MultinomialNB(), 'highest_score': 0.9346036279358358, 'best hyperparameters': {'alpha': 8, 'fit_prior': True}, 'balanceo': 'sin_balanceo', 'segundos': 7.0700297355651855, 'hora_subida': datetime.datetime(2023, 1, 21, 23, 50, 47, 498922)}
{'model_used': GaussianNB(), 'highest_score': 0.7748953409858204, 'best hyperparameters': {'var_smoothing': 2}, 'balanceo': 'sin_balanceo', 'segundos': 7.941832065582275, 'hora_subida': datetime.datetime(2023, 1, 21, 23, 50, 48, 370726)}
{'model_used': BernoulliNB(), 'highest_score': 0.9360901144223224, 'best hyperparameters': {'alpha': 4, 'fit_prior': True}, 'balanceo': 'sin_balanceo', 'segundos': 9.1542017459869

In [16]:

smoteen=MD.modelo_balanceado(SMOTEENN(random_state=RANDOM_STATE), xtr, ytr, models_list, model_hyperparameters)

DATOS DESBALANCEADOS
False    5735
True     1666
Name: cancellation, dtype: int64
DATOS BALANCEADOS
True     4754
False    4189
Name: cancellation, dtype: int64


/home/bdata2/anaconda3/envs/morado_reto06/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1777: UndefinedMetricWarning: 'k' (2) greater than or equal to 'n_classes' (2) will result in a perfect score and is therefore meaningless.
  warnings.warn(
/home/bdata2/anaconda3/envs/morado_reto06/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1777: UndefinedMetricWarning: 'k' (2) greater than or equal to 'n_classes' (2) will result in a perfect score and is therefore meaningless.
  warnings.warn(
/home/bdata2/anaconda3/envs/morado_reto06/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1777: UndefinedMetricWarning: 'k' (2) greater than or equal to 'n_classes' (2) will result in a perfect score and is therefore meaningless.
  warnings.warn(
/home/bdata2/anaconda3/envs/morado_reto06/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1777: UndefinedMetricWarning: 'k' (2) greater than or equal to 'n_classes' (2) will result in a perfect score and is therefore meaningles

{'model_used': LogisticRegression(max_iter=10000), 'highest_score': 1.0, 'best hyperparameters': {'C': 1}, 'segundos': 31.361793279647827, 'balanceo': SMOTEENN(random_state=0), 'hora_subida': datetime.datetime(2023, 1, 22, 10, 17, 48, 955104)}


/home/bdata2/anaconda3/envs/morado_reto06/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1777: UndefinedMetricWarning: 'k' (2) greater than or equal to 'n_classes' (2) will result in a perfect score and is therefore meaningless.
  warnings.warn(
/home/bdata2/anaconda3/envs/morado_reto06/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1777: UndefinedMetricWarning: 'k' (2) greater than or equal to 'n_classes' (2) will result in a perfect score and is therefore meaningless.
  warnings.warn(
/home/bdata2/anaconda3/envs/morado_reto06/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1777: UndefinedMetricWarning: 'k' (2) greater than or equal to 'n_classes' (2) will result in a perfect score and is therefore meaningless.
  warnings.warn(
/home/bdata2/anaconda3/envs/morado_reto06/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1777: UndefinedMetricWarning: 'k' (2) greater than or equal to 'n_classes' (2) will result in a perfect score and is therefore meaningles

{'model_used': MultinomialNB(), 'highest_score': 1.0, 'best hyperparameters': {'alpha': 1, 'fit_prior': True}, 'segundos': 34.094706773757935, 'balanceo': SMOTEENN(random_state=0), 'hora_subida': datetime.datetime(2023, 1, 22, 10, 17, 51, 688019)}


...

{'model_used': GaussianNB(), 'highest_score': 1.0, 'best hyperparameters': {'var_smoothing': 1e-09}, 'segundos': 35.970088481903076, 'balanceo': SMOTEENN(random_state=0), 'hora_subida': datetime.datetime(2023, 1, 22, 10, 17, 53, 563399)}


/home/bdata2/anaconda3/envs/morado_reto06/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1777: UndefinedMetricWarning: 'k' (2) greater than or equal to 'n_classes' (2) will result in a perfect score and is therefore meaningless.
  warnings.warn(
/home/bdata2/anaconda3/envs/morado_reto06/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1777: UndefinedMetricWarning: 'k' (2) greater than or equal to 'n_classes' (2) will result in a perfect score and is therefore meaningless.
  warnings.warn(
/home/bdata2/anaconda3/envs/morado_reto06/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1777: UndefinedMetricWarning: 'k' (2) greater than or equal to 'n_classes' (2) will result in a perfect score and is therefore meaningless.
  warnings.warn(
/home/bdata2/anaconda3/envs/morado_reto06/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1777: UndefinedMetricWarning: 'k' (2) greater than or equal to 'n_classes' (2) will result in a perfect score and is therefore meaningles

{'model_used': BernoulliNB(), 'highest_score': 1.0, 'best hyperparameters': {'alpha': 1, 'fit_prior': True}, 'segundos': 39.451414823532104, 'balanceo': SMOTEENN(random_state=0), 'hora_subida': datetime.datetime(2023, 1, 22, 10, 17, 57, 44724)}


/home/bdata2/anaconda3/envs/morado_reto06/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1777: UndefinedMetricWarning: 'k' (2) greater than or equal to 'n_classes' (2) will result in a perfect score and is therefore meaningless.
  warnings.warn(
/home/bdata2/anaconda3/envs/morado_reto06/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1777: UndefinedMetricWarning: 'k' (2) greater than or equal to 'n_classes' (2) will result in a perfect score and is therefore meaningless.
  warnings.warn(
/home/bdata2/anaconda3/envs/morado_reto06/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1777: UndefinedMetricWarning: 'k' (2) greater than or equal to 'n_classes' (2) will result in a perfect score and is therefore meaningless.
  warnings.warn(
/home/bdata2/anaconda3/envs/morado_reto06/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:1777: UndefinedMetricWarning: 'k' (2) greater than or equal to 'n_classes' (2) will result in a perfect score and is therefore meaningles

KeyboardInterrupt: 

In [19]:

smotetomek=MD.modelo_balanceado(SMOTETomek(random_state=RANDOM_STATE), xtr, ytr, models_list, model_hyperparameters)

DATOS DESBALANCEADOS
False    5735
True     1666
Name: cancellation, dtype: int64
DATOS BALANCEADOS
False    5700
True     5700
Name: cancellation, dtype: int64
{'model_used': LogisticRegression(max_iter=10000), 'highest_score': 0.9137719298245613, 'best hyperparameters': {'C': 10}, 'segundos': 15.713452577590942, 'balanceo': SMOTETomek(random_state=0), 'hora_subida': datetime.datetime(2023, 1, 21, 23, 51, 54, 429385)}
{'model_used': MultinomialNB(), 'highest_score': 0.8337719298245615, 'best hyperparameters': {'alpha': 6, 'fit_prior': True}, 'segundos': 17.061238527297974, 'balanceo': SMOTETomek(random_state=0), 'hora_subida': datetime.datetime(2023, 1, 21, 23, 51, 55, 776900)}
{'model_used': GaussianNB(), 'highest_score': 0.6571929824561403, 'best hyperparameters': {'var_smoothing': 1e-09}, 'segundos': 18.003791093826294, 'balanceo': SMOTETomek(random_state=0), 'hora_subida': datetime.datetime(2023, 1, 21, 23, 51, 56, 719453)}
{'model_used': BernoulliNB(), 'highest_score': 0.82008771

In [20]:
adasyn=MD.modelo_balanceado(ADASYN(random_state=RANDOM_STATE), xtr, ytr, models_list, model_hyperparameters)

DATOS DESBALANCEADOS
False    5735
True     1666
Name: cancellation, dtype: int64
DATOS BALANCEADOS
False    5735
True     5664
Name: cancellation, dtype: int64
{'model_used': LogisticRegression(max_iter=10000), 'highest_score': 0.8259463214860319, 'best hyperparameters': {'C': 10}, 'segundos': 7.956186056137085, 'balanceo': ADASYN(random_state=0), 'hora_subida': datetime.datetime(2023, 1, 21, 23, 52, 55, 333379)}
{'model_used': MultinomialNB(), 'highest_score': 0.696372293172598, 'best hyperparameters': {'alpha': 1, 'fit_prior': False}, 'segundos': 9.027328491210938, 'balanceo': ADASYN(random_state=0), 'hora_subida': datetime.datetime(2023, 1, 21, 23, 52, 56, 404522)}
{'model_used': GaussianNB(), 'highest_score': 0.5827714140551026, 'best hyperparameters': {'var_smoothing': 1e-09}, 'segundos': 9.875614643096924, 'balanceo': ADASYN(random_state=0), 'hora_subida': datetime.datetime(2023, 1, 21, 23, 52, 57, 252807)}
{'model_used': BernoulliNB(), 'highest_score': 0.6996194468180104, 'best

In [21]:

smote=MD.modelo_balanceado(SMOTE(random_state=RANDOM_STATE), xtr, ytr, models_list, model_hyperparameters)

DATOS DESBALANCEADOS
False    5735
True     1666
Name: cancellation, dtype: int64
DATOS BALANCEADOS
False    5735
True     5735
Name: cancellation, dtype: int64
{'model_used': LogisticRegression(max_iter=10000), 'highest_score': 0.9129032258064516, 'best hyperparameters': {'C': 10}, 'segundos': 8.30318808555603, 'balanceo': SMOTE(random_state=0), 'hora_subida': datetime.datetime(2023, 1, 21, 23, 54, 12, 133470)}
{'model_used': MultinomialNB(), 'highest_score': 0.8319093286835223, 'best hyperparameters': {'alpha': 6, 'fit_prior': True}, 'segundos': 9.3342866897583, 'balanceo': SMOTE(random_state=0), 'hora_subida': datetime.datetime(2023, 1, 21, 23, 54, 13, 164568)}
{'model_used': GaussianNB(), 'highest_score': 0.6707933740191805, 'best hyperparameters': {'var_smoothing': 1e-09}, 'segundos': 10.466365337371826, 'balanceo': SMOTE(random_state=0), 'hora_subida': datetime.datetime(2023, 1, 21, 23, 54, 14, 296648)}
{'model_used': BernoulliNB(), 'highest_score': 0.8190061028770707, 'best hype

In [22]:

ENN=MD.modelo_balanceado(EditedNearestNeighbours(),  xtr,ytr,models_list, model_hyperparameters)

DATOS DESBALANCEADOS
False    5735
True     1666
Name: cancellation, dtype: int64
DATOS BALANCEADOS
False    4784
True     1666
Name: cancellation, dtype: int64
{'model_used': LogisticRegression(max_iter=10000), 'highest_score': 0.9386046511627907, 'best hyperparameters': {'C': 1}, 'segundos': 6.065420866012573, 'balanceo': EditedNearestNeighbours(), 'hora_subida': datetime.datetime(2023, 1, 21, 23, 55, 9, 255978)}
{'model_used': MultinomialNB(), 'highest_score': 0.9279069767441861, 'best hyperparameters': {'alpha': 10, 'fit_prior': True}, 'segundos': 7.041749715805054, 'balanceo': EditedNearestNeighbours(), 'hora_subida': datetime.datetime(2023, 1, 21, 23, 55, 10, 232309)}
{'model_used': GaussianNB(), 'highest_score': 0.778139534883721, 'best hyperparameters': {'var_smoothing': 1e-09}, 'segundos': 7.781077861785889, 'balanceo': EditedNearestNeighbours(), 'hora_subida': datetime.datetime(2023, 1, 21, 23, 55, 10, 971635)}
{'model_used': BernoulliNB(), 'highest_score': 0.8857364341085272

In [23]:

nearmiss=MD.modelo_balanceado(NearMiss(), xtr, ytr, models_list, model_hyperparameters)

DATOS DESBALANCEADOS
False    5735
True     1666
Name: cancellation, dtype: int64
DATOS BALANCEADOS
False    1666
True     1666
Name: cancellation, dtype: int64
{'model_used': LogisticRegression(max_iter=10000), 'highest_score': 0.9063603333468402, 'best hyperparameters': {'C': 1}, 'segundos': 1.545865774154663, 'balanceo': NearMiss(), 'hora_subida': datetime.datetime(2023, 1, 21, 23, 55, 25, 335816)}
{'model_used': MultinomialNB(), 'highest_score': 0.8511271391331361, 'best hyperparameters': {'alpha': 3, 'fit_prior': True}, 'segundos': 2.1822168827056885, 'balanceo': NearMiss(), 'hora_subida': datetime.datetime(2023, 1, 21, 23, 55, 25, 972166)}
{'model_used': GaussianNB(), 'highest_score': 0.7231906569237904, 'best hyperparameters': {'var_smoothing': 1e-09}, 'segundos': 2.6096057891845703, 'balanceo': NearMiss(), 'hora_subida': datetime.datetime(2023, 1, 21, 23, 55, 26, 399555)}
{'model_used': BernoulliNB(), 'highest_score': 0.8775517646582115, 'best hyperparameters': {'alpha': 10, 'f

In [24]:

tomeklinks=MD.modelo_balanceado(TomekLinks(), xtr, ytr, models_list, model_hyperparameters)

DATOS DESBALANCEADOS
False    5735
True     1666
Name: cancellation, dtype: int64
DATOS BALANCEADOS
False    5697
True     1666
Name: cancellation, dtype: int64
{'model_used': LogisticRegression(max_iter=10000), 'highest_score': 0.9429580270964314, 'best hyperparameters': {'C': 1}, 'segundos': 7.419516563415527, 'balanceo': TomekLinks(), 'hora_subida': datetime.datetime(2023, 1, 21, 23, 55, 41, 160400)}
{'model_used': MultinomialNB(), 'highest_score': 0.9341295492783142, 'best hyperparameters': {'alpha': 10, 'fit_prior': True}, 'segundos': 8.623372554779053, 'balanceo': TomekLinks(), 'hora_subida': datetime.datetime(2023, 1, 21, 23, 55, 42, 364255)}
{'model_used': GaussianNB(), 'highest_score': 0.7737336366185542, 'best hyperparameters': {'var_smoothing': 1}, 'segundos': 9.347092390060425, 'balanceo': TomekLinks(), 'hora_subida': datetime.datetime(2023, 1, 21, 23, 55, 43, 87976)}
{'model_used': BernoulliNB(), 'highest_score': 0.9354881526904573, 'best hyperparameters': {'alpha': 10, 'f

# Coeficientes

In [144]:
coef=MD.coeficientes_logis(LogisticRegression(max_iter=10000, C=20, random_state=RANDOM_STATE), SMOTEENN(), xtr, ytr)

In [145]:
coef

,columna,coeficiente
20,nieve,-10.646084
24,tormenta,-7.876597
19,niebla,-7.119839
22,otro,-6.985763
16,other,-6.879581
17,lluvia,-6.630566
23,soleado,-6.561200
18,lluvias_fuertes,-6.349083
15,non_refundable,-6.232793
21,nublado,-6.166323


# csv elastic

In [25]:
lista_resultados=[sin_balanceo,smoteen,smotetomek, adasyn, smote, ENN, nearmiss, tomeklinks]

In [26]:
MD.resultados_csv(lista_resultados)

/media/sf_Carpeta_compartida_reto06/Reto_06_Morado/packages/modelado/funciones_data_mining.py:237: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultados=resultados.append(df, ignore_index=True)


In [27]:
df=pd.read_csv(os.path.join('./Datos/Transformados/resultados.csv'))

In [28]:
df

,model_used,highest_score,best hyperparameters,balanceo,segundos,hora_subida
0,LogisticRegression(max_iter=10000),0.943251,{'C': 1},sin_balanceo,6.162464,2023-01-21 23:50:46.591354
1,MultinomialNB(),0.934604,"{'alpha': 8, 'fit_prior': True}",sin_balanceo,7.070030,2023-01-21 23:50:47.498922
2,GaussianNB(),0.774895,{'var_smoothing': 2},sin_balanceo,7.941832,2023-01-21 23:50:48.370726
3,BernoulliNB(),0.936090,"{'alpha': 4, 'fit_prior': True}",sin_balanceo,9.154202,2023-01-21 23:50:49.583092
4,SVC(),0.941630,"{'C': 2, 'kernel': 'rbf'}",sin_balanceo,22.042253,2023-01-21 23:51:02.471143
5,LogisticRegression(max_iter=10000),0.971711,{'C': 20},SMOTEENN(random_state=0),11.043448,2023-01-21 23:51:16.769702
6,MultinomialNB(),0.899921,"{'alpha': 1, 'fit_prior': False}",SMOTEENN(random_state=0),12.147771,2023-01-21 23:51:17.874025
7,GaussianNB(),0.760599,{'var_smoothing': 1e-09},SMOTEENN(random_state=0),13.190036,2023-01-21 23:51:18.916289
8,BernoulliNB(),0.907859,"{'alpha': 1, 'fit_prior': False}",SMOTEENN(random_state=0),15.057152,2023-01-21 23:51:20.783405
9,SVC(),0.975288,"{'C': 3, 'kernel': 'rbf'}",SMOTEENN(random_state=0),29.256371,2023-01-21 23:51:34.982625
